In [115]:
import cmlreaders
from cmlreaders import CMLReader, get_data_index 
import matplotlib.pyplot as plt 
import scipy as sp 
import xarray as xr
from scipy import stats 
from scipy import spatial
from scipy.stats import sem

from sklearn import preprocessing
import os, csv, numpy, pandas
import pandas as pd 
import numpy as np 
import mne
from mne import channels
from mne import time_frequency




import ptsa 
from ptsa.data.TimeSeriesX import TimeSeries 
from ptsa.data.readers import BaseEventReader


%matplotlib inline 
from pylab import *

## BEHAVIORAL

In [116]:
data = get_data_index(kind = 'r1'); data = data[data['experiment'] == 'RepFR1']

sample_evs = sample_reader.load('task_events')

In [117]:
rep_fr = data
repfr_subs = list(rep_fr['subject'].unique())

sessions = np.arange(0,10)
repfr_subs

['R1501J',
 'R1514E',
 'R1516E',
 'R1528E',
 'R1531T',
 'R1547D',
 'R1556J',
 'R1564J',
 'R1568E']

In [118]:
def pullSubInfo(subject, session):
    index = get_data_index(kind = 'r1')
    reader = CMLReader(subject = str(subject), session = int(session), experiment = 'RepFR1')
    evs = reader.load('task_events')
    return evs 

def build_items(evs):
    list_amount = np.arange(0,21)
    ffr_list = 21
    
    rec_evs = evs[evs['type']=='REC_WORD'] #grab recall events 
    word_evs = evs[evs['type']=='WORD'] #word events
    
    word_pool = list(word_evs['item_name'].unique()) 
    total_word_number = len(word_pool)
    
    data_d= {'list_num':list_amount,
          'ffr_list':ffr_list,
          'wordpool':word_pool,
          'total_words':total_word_number,
          'rec_evs':rec_evs,
          'word_evs':word_evs}

    return data_d


def Recall_A(data_d):
    list_amount = data_d['list_num']
    ffr_list = data_d['ffr_list']
    word_pool = data_d['wordpool']
    total_word_number = data_d['total_words']
    rec_evs = data_d['rec_evs']
    word_evs = data_d['word_evs']
    
    repeats = [1, 2, 3]
    r1 = []
    r2 = []
    r3 = []
    
    all_recalls = []
    for l in list_amount:
        rep_d = {'1':[],
                '2': [],
                '3': []}
        
        d_list = word_evs[word_evs['list']==l]
        rec_list = rec_evs[rec_evs['list']==l]

        #c_recs = d_list[d_list['recalled']==True]


        rec_words = rec_list['item_name'].unique()
        all_recalls.extend(rec_words)


        for r in repeats:

            l_name = 'r'+str(r)
            repeat_list = d_list[d_list['repeats']==r]

            rep_d[str(r)] = repeat_list['item_name'].unique()

        r1.extend(rep_d['1'])
        r2.extend(rep_d['2'])
        r3.extend(rep_d['3'])

    prec = len(all_recalls)/total_word_number

    rec1 = [a for a in all_recalls if a in r1]
    rec2 = [a for a in all_recalls if a in r2]
    rec3 = [a for a in all_recalls if a in r3]

    prec_r1 = len(rec1)/len(r1)
    prec_r2 = len(rec2)/len(r2)
    prec_r3 = len(rec3)/len(r3)
    
    
    num_r_words = [rec1, rec2, rec3]
    prec_repeats = [prec, prec_r1, prec_r2, prec_r3]
    
    
    d_ = {'words':num_r_words,
         'probs': prec_repeats}
    
    return d_
        

In [119]:
all_data = []
full_data = []
# Put Subject Id Here
subject = 'R1501J'
for sess in sessions:
    filter_c = False
    try:
        sub_info = pullSubInfo(subject, sess)
        data_d = build_items(sub_info)
        recall_data = Recall_A(data_d)


        full_sd = ['Recall Rate (%)', sess,recall_data['probs'][0], recall_data['probs'][1], recall_data['probs'][2], recall_data['probs'][3]]
        full_data.append(full_sd)
        for value in recall_data['probs']:
            if value <=0.50:
                filter_c = True


            else:
                pass 

        if filter_c != True:

            sub_ = [subject, sess,recall_data['probs'][0], recall_data['probs'][1], recall_data['probs'][2], recall_data['probs'][3]]
            #print(sub_)
            sub_data.append(sub_)


            #sub_data.append(recall_data['probs'])
        elif filter_c == True:
            pass 


    except:
        count=0


all_data.extend(sub_data)



In [120]:
sub_df = pd.DataFrame(data = full_data, columns = ['Recall Type','session','Total Recall', 'Rep1','Rep2','Rep3'])

sub_df = sub_df.groupby('Recall Type').mean().drop('session', axis = 1).round(4)*100
sub_df = sub_df.transpose()
sub_df

Recall Type,Recall Rate (%)
Total Recall,48.76
Rep1,22.22
Rep2,47.09
Rep3,65.87


In [114]:
sub_l = list(sub_df['subject'].unique())
for s in sub_l:
    print(sub_df[sub_df['subject']==s])


KeyError: 'subject'

## EEG

In [24]:
def get_events(subject, session):
    reader = CMLReader(subject = subject, experiment = 'ltpRepFR', session = session)
    return reader.load('task_events'), reader

def generate_events(task_events):
    task_events = task_events 
    
    recall = task_events.query("type =='REC_WORD'")
    im_recall = recall[recall['list']!=26]
    de_recall = recall[recall['list']==26]
    
    im_r = im_recall.to_records(index = False)
    de_r = de_recall.to_records(index = False)
    
    return im_recall, de_recall, im_r, de_r 

def get_eeg(reader, subject, session, events, rel_start = 0, rel_stop = 500, filter_data = False):
    rel_start = rel_start
    rel_stop = rel_stop
    
    reader = reader
    
    eeg = reader.load_eeg(events = events, rel_start = rel_start, rel_stop = rel_stop, clean = False).to_mne()
    
    if filter_data == True:
        eeg = eeg.filter(l_freq = 62., h_freq = 58., method = 'iir')
        eeg = eeg.filter(l_freq = 122., h_freq = 118., method = 'iir')
    
    elif filter_data == False:
        pass 
    
    channels = eeg.ch_names
    
    return eeg, channels

def get_powers(eeg, fmin, fmax, channels, event_record):
    
    power, fdone = mne.time_frequency.psd_multitaper(eeg, fmin = fmin, fmax = fmax, verbose = False)
    
    p = TimeSeries(power,
                   dims = ('events', 'channels', 'frequency'),
                   coords = {'events': event_record, 'channels': channels[:128], 'frequency': fdone, 'samplerate':0})
                   
                   
                   
                
    
    
    return p, power, fdone


def split_ROIs(power, sys):

    mode = 8

    if mode==8:
        # by eight rois
        ROI_list = ROI_LIST

        power_byroi = []
        for roi in ROI_list:
            channels = ROIs[sys][roi]
            x = power.sel(channels=np.in1d(power.channels.values, channels)).mean('channels')
            power_byroi.append(x)

        power_byroi = xr.concat(power_byroi, dim='ROIs')
        power_byroi.coords['ROIs'] = ROI_list

    elif mode==2:
        # by front and back
        ROI_list = ['anterior','posterior']

        power_byroi = []
        for roi in ROI_list:
            if roi=='anterior':
                channels = np.hstack((ROIs[sys]['LAS'],ROIs[sys]['LAI'],ROIs[sys]['RAS'],ROIs[sys]['RAI'])).tolist()
            elif roi=='posterior':
                channels = np.hstack((ROIs[sys]['LPS'],ROIs[sys]['LPI'],ROIs[sys]['RPS'],ROIs[sys]['RPI'])).tolist()
            x = power.sel(channels=np.in1d(power.channels.values, channels)).mean('channels')
            power_byroi.append(x)

        power_byroi = xr.concat(power_byroi, dim='ROIs')
        power_byroi.coords['ROIs'] = ROI_list

    return power_byroi


def power_by_roi(power, sys, log = True):
    
    power_split = split_ROIs(power, sys)
    
    if log == True:
        power_split = np.log10(np.mean(power_split,1))

    elif log == False:
        pass 
    
    
    return power_split 

def plot_spectrum(power_data, plot_title, save_title, save = True):
    ax = plt.subplot(111)
    plt.imshow(power_data, interpolation = 'none', cmap = 'coolwarm', extent = [4, 128, 80, 0])
    plt.ylabel('ROIs')
    plt.xlabel('log frequency')
    plt.title(plot_title)
    ax.set_yticklabels(['LAI','RAI','LAS','RAS','LPI','RPI','LPS','RPS'])
    ax.set_yticks([5, 15, 25, 35, 45, 55, 65, 75])
    ax.set_xticks([4, 12,  20,  28,  36,  44,
            52,  60,  68,  76,  84,  92, 100,
           112, 124])
    
    plt.tight_layout()
    
    
    if save == True:
        plt.savefig('repfr_figs/'+str(save_title)+'.png')
        
    elif save == False:
        pass 
    
       
    return
    

   


    
    

In [25]:
subjects = ['LTP441, LTP442, LTP443, LTP444, LTP445, LTP446, LTP447']
sessions = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

freqs= [  2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,  22.,
        24.,  26.,  32.,  38.,  44.,  50.,  56.,  62.,  68.,  74.,  80.,
        86.,  92.,  98., 104., 110., 116., 122., 128.]

In [26]:
 #roi_text = ['LAI','RAI','LAS','RAS','LPI','RPI','LPS','RPS']
ROI_LIST = ['LAS','LAI','LPS','LPI','RAS','RAI','RPS','RPI']
ROIs = {'egi':{'LAS':['E12','E13','E19','E20','E24','E28','E29'],
               'LAI':['E23','E26','E27','E33','E34','E39','E40'],
               'LPS':['E37','E42','E52','E53','E54','E60','E61'],
               'LPI':['E51','E58','E59','E64','E65','E66','E69'],
               'RAS':['E4','E5','E111','E112','E117','E118','E124'],
               'RAI':['E2','E3','E109','E115','E116','E122','E123'],
               'RPS':['E78','E79','E85','E86','E87','E92','E93'],
               'RPI':['E84','E89','E90','E91','E95','E96','E97']},
        'biosemi':{'LAS':['C24','C25','D2','D3','D4','D11','D12','D13'],
                   'LAI':['C31','C32','D5','D6','D9','D10','D21','D22'],
                   'LPS':['D29','A5','A6','A7','A8','A17','A18'],
                   'LPI':['D30','D31','A9','A10','A11','A15','A16'],
                   'RAS':['B30','B31','B32','C2','C3','C4','C11','C12'],
                   'RAI':['B24','B25','B28','B29','C5','C6','C9','C10'],
                   'RPS':['A30','A31','A32','B3','B4','B5','B13'],
                   'RPI':['A28','A29','B6','B7','B8','B11','B12']}}


ROI_order = ['LAS', 'LAI', 'LPS', 'LPI', 'RAS', 'RAI', 'RPS', 'RPI']

In [ ]:
sample_evs, evs_reader  = get_events('LTP442', 6)
im_recall_o, de_recall_o, im_r, de_r = generate_events(sample_evs)

im_recall, im_channels = get_eeg(evs_reader,'LTP442', 6, im_recall_o, filter_data = True)
de_recall, de_channels = get_eeg(evs_reader,'LTP442', 6, de_recall_o, filter_data = True)
#can condence this into one eeg function by creating a mask for all recall events before and during lsit 26 (EFR)

im_p, im_power, fdone =  get_powers(im_recall, freqs[0], freqs[-1], im_channels, im_r)
de_p, de_power, _ = get_powers(de_recall, freqs[0], freqs[-1], de_channels, de_r)

im_split = power_by_roi(im_p, 'biosemi')
de_split = power_by_roi(de_p, 'biosemi')

difference = (im_split - de_split)

Extracting EDF parameters from /protocols/ltp/subjects/LTP442/experiments/ltpRepFR/sessions/6/ephys/current_processed/LTP442_session_6.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9377791  =      0.000 ...  4579.000 secs...
Not setting metadata
Not setting metadata
304 matching events found
No baseline correction applied
0 projection items activated
Loading data for 304 events and 1025 original time points ...
0 bad epochs dropped
Not setting metadata
Not setting metadata
304 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Setting up band-stop filter from 58 - 62 Hz

IIR filter parameters
---------------------
Butterworth bandstop zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 58.00, 62.00 Hz: -6.02, -6.02 dB

Setting up band-stop filter from 1.2e+02 - 1.2e+02 Hz

IIR filter parameters
--------

In [ ]:
plot_spectrum(difference, 'Delayed vs. Immediate Recall', 'LTP422.6')